In [1]:
# Import Taichi
import taichi as ti
import taichi.math as tm
import numpy as np
import sys
import time

[Taichi] version 1.7.0, llvm 15.0.1, commit 2fd24490, win, python 3.10.6


In [2]:
ti.init(arch=ti.vulkan)

# type aliases
vec2 = tm.vec2
vec3 = tm.vec3
vec2i = ti.types.vector(2, int)

[Taichi] Starting on arch=vulkan


In [3]:
# define resolution
WIDTH = 800
HEIGHT = 800

# configurations
PARTICLE_COUNT = 100 # define shape
PARTICLE_RADIUS = 0.004 # for rendering only
USE_GRAVITY = True
GRAVITY = vec3(0.0, 0.0005, 0.0)

# for particle simulation only
CONNECTION_COUNT = PARTICLE_COUNT // 2

# for cloth simulation only
GRID_SIZE = vec2i(20, 20)
PARTICLE_COUNT = GRID_SIZE.x * GRID_SIZE.y
GAP = vec2(1 / GRID_SIZE.x, 1 / GRID_SIZE.y)
OFFSET = vec2(0.025, 0.025)

STRUCTURE_CONNECTION_COUNT = (GRID_SIZE.x - 1) * GRID_SIZE.x + (GRID_SIZE.y - 1) * GRID_SIZE.y
SHEAR_CONNECTION_COUNT = (GRID_SIZE.x - 1) * (GRID_SIZE.y - 1) * 2
FLEXION_CONNECTION_COUNT = (GRID_SIZE.x * (GRID_SIZE.y - 2)) + (GRID_SIZE.y * (GRID_SIZE.x - 2))
CONNECTION_COUNT = STRUCTURE_CONNECTION_COUNT + SHEAR_CONNECTION_COUNT + FLEXION_CONNECTION_COUNT
STIFFNESS = [0.8, 0.2, 0.1]
# STIFFNESS = [0.8, 0.3, 0.15]

# define variables - general
masses = ti.field(dtype=float, shape=PARTICLE_COUNT)
positions = ti.Vector.field(3, dtype=float, shape=PARTICLE_COUNT)
previous_positions = ti.Vector.field(3, dtype=float, shape=PARTICLE_COUNT)
pinned = ti.field(ti.u8, shape=PARTICLE_COUNT)

# variables - springs
rest_lengths = ti.field(ti.f32, shape=CONNECTION_COUNT)
connection_starts = ti.field(ti.i32, shape=CONNECTION_COUNT)
connection_ends = ti.field(ti.i32, shape=CONNECTION_COUNT)
stiffness_amounts = ti.field(ti.f32, shape=CONNECTION_COUNT)

connection_index = ti.field(dtype=ti.i32, shape=(1,))

# variables - springs - render
indices = ti.field(int, shape=SHEAR_CONNECTION_COUNT * 3)
colors = ti.Vector.field(3, dtype=float, shape=PARTICLE_COUNT)

# variable - cloth - ball center
collision_ball_radius = 0.1
collision_ball_center = ti.Vector.field(3, dtype=float, shape=(2, ))
collision_ball_center[0] = [0.5, 0.5, 1]
collision_ball_center[1] = [0.5, 0.75, 2]

In [4]:
# Utility Functions
@ti.func
def generate_random_float(minInclude, maxInclude):
    random_float = minInclude + (maxInclude - minInclude) * ti.random()
    return random_float

@ti.func
def random_position() -> vec3:
    x = generate_random_float(-0.25, 0.25)
    y = generate_random_float(-0.25, 0.25)
    return vec3(x, y, 0)

@ti.func
def random_velocity(minInclude: vec2, maxInclude: vec2) -> vec3:
    x = generate_random_float(minInclude.x, maxInclude.x)
    y = generate_random_float(minInclude.y, maxInclude.y)
    return vec3(x, y, 0)

@ti.func
def grid_position(row: int, col: int) -> vec3:
    x = 0 + col * GAP.x
    y = 0 + row * GAP.y
    return vec3(x, y, 0)

@ti.func
def hsv_to_rgb(h, s, v):
    r, g, b = 0.0, 0.0, 0.0
    if s == 0.0:
        r = v
        g = v
        b = v
    else:
        i = int(h * 6.0)
        f = (h * 6.0) - i
        p = v * (1.0 - s)
        q = v * (1.0 - s * f)
        t = v * (1.0 - s * (1.0 - f))
        i = i % 6

        if i == 0:
            r, g, b = v, t, p
        elif i == 1:
            r, g, b = q, v, p
        elif i == 2:
            r, g, b = p, v, t
        elif i == 3:
            r, g, b = p, q, v
        elif i == 4:
            r, g, b = t, p, v
        else:
            r, g, b = v, p, q

    return r, g, b

In [5]:
@ti.func
def init_connections():
    for i in range(CONNECTION_COUNT):
        connection_starts[i] = i * 2
        connection_ends[i] = i * 2 + 1

@ti.func
def init_positions_cloth():
    for row in range(GRID_SIZE.x):
        for col in range(GRID_SIZE.y):
            x = OFFSET.x + col * GAP.x
            y = OFFSET.y + row * GAP.y
            index = row * GRID_SIZE.y + col
            positions[index] = ti.Vector([x, y, 0.0])
            if row == GRID_SIZE.x - 1 : pinned[index] = 1
            # if row == GRID_SIZE.x - 1 and (col == 0 or col == (GRID_SIZE.y - 1) // 2 or col == GRID_SIZE.y - 1): pinned[index] = 1
            
@ti.func
def init_connections_cloth():
    connection_index[0] = 0
    
    # Horizontal constraints
    for row in range(GRID_SIZE[0]):
        for col in range(GRID_SIZE[1] - 1):
            index = row * GRID_SIZE[1] + col
            idx = ti.atomic_add(connection_index[0], 1)
            connection_starts[idx] = index
            connection_ends[idx] = index + 1
            stiffness_amounts[idx] = STIFFNESS[0]
    
    # Vertical constraints
    for row in range(GRID_SIZE[0] - 1):
        for col in range(GRID_SIZE[1]):
            index = row * GRID_SIZE[1] + col
            idx = ti.atomic_add(connection_index[0], 1)
            connection_starts[idx] = index
            connection_ends[idx] = index + GRID_SIZE[1]
            stiffness_amounts[idx] = STIFFNESS[0]
            
    # Shear constraints - Diagonal Right (\ direction)
    for row in range(GRID_SIZE[0] - 1):
        for col in range(GRID_SIZE[1] - 1):
            index = row * GRID_SIZE[1] + col
            idx = ti.atomic_add(connection_index[0], 1)
            connection_starts[idx] = index
            connection_ends[idx] = index + GRID_SIZE[1] + 1
            stiffness_amounts[idx] = STIFFNESS[1]
    
    # Shear constraints - Diagonal Left (/ direction)
    for row in range(1, GRID_SIZE[0]):
        for col in range(GRID_SIZE[1] - 1):
            index = row * GRID_SIZE[1] + col
            idx = ti.atomic_add(connection_index[0], 1)
            connection_starts[idx] = index
            connection_ends[idx] = index - GRID_SIZE[1] + 1
            stiffness_amounts[idx] = STIFFNESS[1]
            
    # Flexion constraints - Horizontal
    for row in range(GRID_SIZE[0]):
        for col in range(GRID_SIZE[1] - 2):
            index = row * GRID_SIZE[1] + col
            idx = ti.atomic_add(connection_index[0], 1)
            connection_starts[idx] = index
            connection_ends[idx] = index + 2  
            stiffness_amounts[idx] = STIFFNESS[2]

    # Flexion constraints - Vertical
    for row in range(GRID_SIZE[0] - 2):
        for col in range(GRID_SIZE[1]):
            index = row * GRID_SIZE[1] + col
            idx = ti.atomic_add(connection_index[0], 1)
            connection_starts[idx] = index
            connection_ends[idx] = index + GRID_SIZE[1] * 2
            stiffness_amounts[idx] = STIFFNESS[2]

@ti.func  
def initialize_mesh_indices():
    for i, j in ti.ndrange(GRID_SIZE.x - 1, GRID_SIZE.y - 1):
        n = GRID_SIZE.y
        quad_id = (i * (n - 1)) + j
        # 1st triangle of the square
        indices[quad_id * 6 + 0] = i * n + j
        indices[quad_id * 6 + 1] = (i + 1) * n + j
        indices[quad_id * 6 + 2] = i * n + (j + 1)
        # 2nd triangle of the square
        indices[quad_id * 6 + 3] = (i + 1) * n + j + 1
        indices[quad_id * 6 + 4] = i * n + (j + 1)
        indices[quad_id * 6 + 5] = (i + 1) * n + j
    
    for i, j in ti.ndrange(GRID_SIZE.x, GRID_SIZE.y):
        n = GRID_SIZE.y
        hue = ((i / GRID_SIZE.x) + (j / GRID_SIZE.y)) / 2
        saturation = 0.5
        value = 0.9
        r, g, b = hsv_to_rgb(hue, saturation, value)
        colors[i * n + j] = (r, g, b)

@ti.func
def init_rest_lengths():
    for i in range(CONNECTION_COUNT):
        start_index = connection_starts[i]
        end_index = connection_ends[i]
    
        particle_start = positions[start_index]
        particle_end = positions[end_index]
        vector = particle_end - particle_start
        magnitude = vector.norm()

        rest_lengths[i] = magnitude

@ti.kernel
def init():
    # particle simulations
    # for i in range(PARTICLE_COUNT):
    #     positions[i] = random_position()
    #     previous_positions[i] = positions[i] - random_velocity(vec2(-0.05), vec2(0.05))
    # init_connections()
    # init_rest_lengths()
    
    # cloth simulations
    init_positions_cloth()
    init_connections_cloth()
    init_rest_lengths()
    initialize_mesh_indices()

In [6]:
@ti.kernel
def update():
    for i in range(PARTICLE_COUNT):
        if pinned[i] == 1: continue
        current_position = positions[i]
        # verlet simulation
        next_position = 2.0 * current_position - previous_positions[i]
        if USE_GRAVITY: next_position -= GRAVITY
        
        positions[i] = next_position
        previous_positions[i] = current_position

In [7]:
@ti.kernel
def update_collision_ball(time_passed: float):
    radius = 0.25
    x_movement = ti.cos(time_passed) * radius + 0.5
    z_movement = ti.sin(time_passed) * radius

    collision_ball_center[0].x = x_movement
    collision_ball_center[0].z = z_movement
    # collision_ball_center[1].x = x_movement
    # collision_ball_center[1].z = -z_movement

In [8]:
@ti.func
def collision_resolution():
    for i, j in ti.ndrange(PARTICLE_COUNT, collision_ball_center.shape[0]):
        offset_to_center = positions[i] - collision_ball_center[j]
        distance = offset_to_center.norm()
    
        if distance <= collision_ball_radius:
            extra_offset = 0.01
            normalized_direction = offset_to_center.normalized()
            contact_point = collision_ball_center[j] + normalized_direction * (collision_ball_radius + extra_offset)
            
            previous_positions[i] = positions[i]
            positions[i] = contact_point

@ti.func
def apply_shape_constraints():
    for i in range(CONNECTION_COUNT):
        # gain indicies
        start_index = connection_starts[i]
        end_index = connection_ends[i]
        if start_index == -1 or end_index == -1: continue
        # gain particle position value
        particle_start = positions[start_index]
        particle_end = positions[end_index]
        # calculate offset
        vector = particle_start - particle_end
        current_length = vector.norm()
        difference = rest_lengths[i] - current_length
        percentage = difference / current_length / 2
        # stifness_index = stiffness_amounts[i]
        offset = vector * percentage * stiffness_amounts[i]
        # apply to real value
        if pinned[start_index] == 0: positions[start_index] += offset * 0.5
        if pinned[end_index] == 0: positions[end_index] -= offset * 0.5

@ti.func
def apply_boundary_constranits():
    for i in range(PARTICLE_COUNT):
         # boundary constraints for x-axis
        if positions[i].x > 1:
            previous_positions[i].x = positions[i].x
            positions[i].x = 1
        elif positions[i].x < 0:
            previous_positions[i].x = positions[i].x 
            positions[i].x = 0

        # boundary constraints for y-axis
        if positions[i].y > 1:
            previous_positions[i].y = positions[i].y
            positions[i].y = 1
        elif positions[i].y < 0:
            previous_positions[i].y = positions[i].y
            positions[i].y = 0

@ti.kernel
def apply_constriants():
    collision_resolution()
    apply_shape_constraints()
    apply_boundary_constranits()

In [9]:
init()

window = ti.ui.Window("Mass Spring System", (WIDTH, HEIGHT), vsync=True)
# canvas to render a scene
canvas = window.get_canvas()
canvas.set_background_color((0, 0, 0))

# Setting up the camera
scene = window.get_scene()
camera = ti.ui.Camera()
# camera.position(0.5, 0.5, 1.25)
camera.position(0.5, 0.5, 1.75)
camera.lookat(0.5, 0.5, 0) 
camera.up(0, 1, 0) 
scene.set_camera(camera)

gui = window.get_gui()

boundaries = ti.Vector.field(3, dtype=float, shape=(4, ))
boundaries[0] = [0, 0, 0]
boundaries[1] = [1, 0, 0]
boundaries[2] = [0, 1, 0]
boundaries[3] = [1, 1, 0]

static_sphere = ti.Vector.field(3, dtype=float, shape=(1, ))
static_sphere[0] = [0, 0, 0]

special_line = ti.Vector.field(3, dtype=float, shape=(2, ))

start_time = time.time()
previous_time = time.time()

RESTART_CLICKED = False

while window.running:
    # update delta time
    time_passed = time.time() - start_time
    current_time = time.time()
    delta_time = current_time - previous_time
    previous_time = current_time
    
    if(RESTART_CLICKED): init()
    
    scene.point_light(pos=(0, 1, 2), color=(0.5, 0.5, 0.5))
    scene.ambient_light((0.5, 0.5, 0.5))
    
    # simulation update
    update()
    update_collision_ball(time_passed)
    apply_constriants()
    
    # simulation rendering
    scene.mesh(positions, indices=indices, per_vertex_color=colors, two_sided=True)
    
    scene.particles(positions, radius=PARTICLE_RADIUS, color=(0.95, 0.95, 0.95))
    scene.particles(boundaries, radius=PARTICLE_RADIUS, color=(0, 1, 0))
    scene.particles(static_sphere, radius=PARTICLE_RADIUS, color=(1, 0, 0))

    scene.particles(collision_ball_center, radius=collision_ball_radius * 0.75, color=(0.75, 0.75, 0.75))
    
    # scene.lines(positions, color = (0.75, 0.05, 0), width = 2.0)
    
    # special_line[0] = positions[0]
    # special_line[1] = positions[1]
    # scene.lines(special_line, color = (1, 0.5, 0), width = 3.0)
    # scene.particles(special_line, radius=PARTICLE_RADIUS, color=(0.75, 0.75, 0.75))
    
    with gui.sub_window("Configurations", x=0.02, y=0.02, width=0.3, height=0.25):
        RESTART_CLICKED = gui.button("Restart")
        USE_GRAVITY = gui.checkbox("Enable Gravity", USE_GRAVITY)
        gui.text("Stiffness")
        STIFFNESS[0] = gui.slider_float("Sturcture", STIFFNESS[0], minimum=0, maximum=1)
        STIFFNESS[0] = gui.slider_float("Shear", STIFFNESS[0], minimum=0, maximum=1)

    # flush
    canvas.scene(scene)
    window.show()